# Mixtral in Colab

Welcome! In this notebook you can run [Mixtral8x7B-Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) with decent generation speed **right in Google Colab or on a consumer-grade GPU**. This was made possible by quantizing the original model in mixed precision and implementing a MoE-specific offloading strategy.

To learn more, read our [tech report](https://arxiv.org/abs/2312.17238) or check out the [repo](https://github.com/dvmazur/mixtral-offloading) on GitHub.

One will need approximately 16 GB of VRAM and 11 GB of RAM to run this notebook and generate somewhat long texts.


<details>

<summary>How to balance between RAM and GPU VRAM usage</summary>

You can balance between RAM and GPU VRAM usage by changing <code>offload_per_layer</code> variable in the <a href="#scrollTo=_mIpePTMFyRY&line=10&uniqifier=1">Initialize model</a> section. Increasing <code>offload_per_layer</code> will decrease GPU VRAM usage, increase RAM usage and decrease generation speed. Decreasing <code>offload_per_layer</code> will have the opposite effect.

Note that this notebook should run normally in Google Colab with <code>offload_per_layer = 4</code>, but may crush with other values. However, if you run this somewhere else, you're free to play with this variable.
</details>

## Install and import libraries

In [ ]:
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/type-shangshu/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

/bin/bash: line 1: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
/bin/bash: line 1: ldconfig: command not found


In [1]:
import sys
import os

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root: {project_root}")

# Add the project root to Python path
sys.path.insert(0, project_root)

# Now import other dependencies
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

# Import project specific modules
from src.build_model import OffloadConfig, QuantConfig, build_model

Project root: /home/ljw/mixtral-offloading
hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Login your huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Initialize model

In [ ]:
# Initialize model parameters
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
# offload_per_layer = 4
offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

##### Change Cache Strategy as you want #####
cache_strategy = "random" #Options: "lru", "lfu", "random"

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
    cache_strategy=cache_strategy,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/torch/nn/init.py:582: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|██████████| 32/32 [00:44<00:00,  1.40s/it]


## Specify cache strategy

In [3]:
new_strategy = "lru" #options: "lru", "lfu", "random"

## Run the model

In [ ]:
from transformers import AutoTokenizer, TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

chat_history = []

def switch_cache_strategy(model, new_strategy: str):
    """Switch cache strategy during inference"""
    for layer in model.model.layers:
        if hasattr(layer, 'block_sparse_moe'):
            if hasattr(layer.block_sparse_moe, 'expert_cache'):
                layer.block_sparse_moe.expert_cache.switch_cache_strategy(new_strategy)
    print(f"Cache strategy switched to: {new_strategy}")

switch_cache_strategy(model, new_strategy)

while True:
    print("User: ", end="")
    user_input = input()
    print(f"{user_input}\n")

    chat_history.append(dict(role="user", content=user_input))

    input_ids = tokenizer.apply_chat_template(chat_history, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids)

    print("Mixtral: ", end="")
    result = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        streamer=streamer,  
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_hidden_states=False,
    )
    print("\n")

    output_ids = result["sequences"][0]
    reply = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True)
    
    chat_history.append(dict(role="assistant", content=reply))

Cache strategy switched to: lru
User: how does lrucache work

Mixtral: LRU (Least Recently Used) Cache is a cache optimization algorithm that works by evicting the least recently used data or keys, i.e., the ones that haven’t been accessed in a while. LRU Cache is a data structure that uses an internal cache algorithm to improve performance by keeping frequently used data easy to access.

Here's how an LRU Cache works:

1. When initializing an LRU Cache, it's usually implemented with a fixed-size data structure, such as an array, a hash table, or a linked list, that can hold a fixed number of elements.
2. When a new element is added to the cache, the LRU Cache checks if the new element already exists in the cache.
3. If the new element is already in the cache, the LRU Cache updates the access time or usage counter for that element to indicate that it has been recently accessed.
4. If the new element is not already in the cache, the LRU Cache adds it to the cache, and if the cache is fu

KeyboardInterrupt: Interrupted by user